# TensorFlow Introduction

Dale Smith, Ph.D. Math, Georgia Tech.

Data Scientist with FraudScope.

https://github.com/dtsmith2001

## Preliminaries

I am using Anaconda 4.3.1. I ran

    conda update --yes conda
    conda update --all --yes

TensorFlow can be installed via conda-forge but it's version 0.12. Use

    pip install tensorflow

For GPU use, follow the [instructions](https://www.tensorflow.org/install/), or use an AWS instance that is already configured. You can also build your own instance; see [http://expressionflow.com/2016/10/09/installing-tensorflow-on-an-aws-ec2-p2-gpu-instance/](http://expressionflow.com/2016/10/09/installing-tensorflow-on-an-aws-ec2-p2-gpu-instance/) but note this uses an older version of TensorFlow.

I also installed [jupyter-themer](https://github.com/transcranial/jupyter-themer), which is why my Pandas tables have every other row grey.

## TensorFlow

TensorFlow was created internally at Google to serve as their AI platform for deep learning, but it does more than deep learning or neural networks. We will use it to train a linear regression model.

[Keras](https://keras.io) was recently adopted by Google as their neural network front-end. I highly recommend using Keras instead of TensorFlow to build a neural network.

Many people believe Google open sourced TensorFlow because they realized the data is more valuable than the code itself.

## Declarative versus Imperative Programming

With **declarative programming** you tell the computer what you want to do, but not how to do it. This is what you do when you write an SQL query or a program in TensorFlow.

**Imperative programming** requires you to tell the computer how to do the job (Python, C/C++, C#, etc).

## Let's get started!

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
hello = tf.constant('Hello, TensorFlow!')
hello

<tf.Tensor 'Const:0' shape=() dtype=string>

In [3]:
sess = tf.Session()

**Note**: Nothing happens until you run the session.

In [4]:
print(sess.run(hello))

b'Hello, TensorFlow!'


## Computational Graphs

Yes, these are real graph structures. I would describe them as acyclic directed graphs. These structures are often used to find best paths to recompute, say, bond prices.

A **computational graph** is a series of operations (+, -, etc) expressed as nodes of the graph. Each node takes a tensor as input and a tensor as output.

A **[tensor](https://en.wikipedia.org/wiki/Tensor)** is an object represented as a multidimensional array with respect to a basis, much as a matrix is a representation of a **linear transformation** with respect to a basis. The Stress tensor and curvature tensor are used extensively in elastic materials and general relativity. Tensors were also used in psychology to handle multidimensional data, and more recently in machine learning.

### Create Nodes and Visualize a Simple Computation Graph

We can create a constant node as

In [5]:
node1 = tf.constant(3.0)

In [6]:
node2 = tf.constant(4.0)

In [7]:
node2

<tf.Tensor 'Const_2:0' shape=() dtype=float32>

Add two objects:

In [8]:
node3 = tf.add(node1, node2)
node3

<tf.Tensor 'Add:0' shape=() dtype=float32>

In [9]:
sess = tf.Session()

In [10]:
print("sess.run(node3): ", sess.run(node3))

sess.run(node3):  7.0


In [11]:
node4 = tf.constant(10.0)

In [12]:
node5 = tf.add(node3, node4)

TensorFlow comes with an application called **TensorBoard** which we can use to visualize the computational graph for the nodes we've constructed.

In [15]:
graph = tf.get_default_graph()
summary_writer = tf.summary.FileWriter("/Users/dtsmith/Temp", graph)
summary_writer.flush()

Now run ```tensorboard --logdir=/Users/driver.dan12/Temp``` and click on http://localhost:6006. Here is the computation graph ![graph1.png](graph1.png).

## Warnings from TensorFlow

> ```The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.```

Compile your own TensorFlow. See http://stackoverflow.com/questions/42270739/how-do-i-resolve-these-tensorflow-warnings/42975902#42975902.

Let's clear the default graph and take a look at this more closely.

In [16]:
tf.reset_default_graph()

## Does this look odd?

In [17]:
x = tf.Variable(0, name='x')
model = tf.global_variables_initializer()
with tf.Session() as session:
    for i in range(5):
        session.run(model)
        x = x + 1
        print(session.run(x))

1
2
3
4
5


In [18]:
x = tf.Variable(0, name='x')
model = tf.global_variables_initializer()
with tf.Session() as session:
    for i in range(5):
        x = x + 1
        session.run(model)
        print(session.run(x))

1
2
3
4
5


Nope, this makes perfect sense - we are not executing the computation graph until the ```run``` command. So whether we put the ```x = x + 1``` before or after ```session.run(model)```. This statement initializes ```x``` to be zero, and ```run``` executes the computation graph built so far.

But what does ```global_variables_initializer``` do?

It initializes all the *global variables* in the graph. The global variables are shared across machines in a distributed environment. Contrast with *local variables* which are per-process variables.

We don't need to initialize ```Constant```, just ```Variable```. Reflect on that a little bit.

In [17]:
graph = tf.get_default_graph()
summary_writer = tf.summary.FileWriter("/Users/driver.dan12/Temp", graph)
summary_writer.flush()

Here is the computation graph ![graph2.png](graph2.png)

In [19]:
tf.reset_default_graph()

## Matrix Multiplication

In [20]:
W = tf.Variable(tf.random_uniform([1000,1000]))
x = tf.Variable(tf.ones([1000,1]))
sess = tf.Session()

In [21]:
sess.run(tf.matmul(W, x))

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: Variable/read = Identity[T=DT_FLOAT, _class=["loc:@Variable"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable)]]

Caused by op 'Variable/read', defined at:
  File "//anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "//anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "//anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "//anaconda/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "//anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "//anaconda/envs/tensorflow/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, 

In [22]:
sess.run(tf.global_variables_initializer())

In [23]:
sess.run(tf.matmul(W, x))

array([[ 513.67773438],
       [ 486.7638855 ],
       [ 504.49401855],
       [ 499.72555542],
       [ 495.87896729],
       [ 511.04278564],
       [ 492.54116821],
       [ 519.60455322],
       [ 515.26721191],
       [ 507.90615845],
       [ 494.32861328],
       [ 486.29190063],
       [ 505.69464111],
       [ 496.29101562],
       [ 503.07104492],
       [ 502.17071533],
       [ 517.3918457 ],
       [ 500.72662354],
       [ 496.21710205],
       [ 519.97351074],
       [ 498.82116699],
       [ 496.81161499],
       [ 502.06903076],
       [ 514.77770996],
       [ 515.63720703],
       [ 482.14489746],
       [ 505.35287476],
       [ 502.90933228],
       [ 506.31072998],
       [ 501.13769531],
       [ 504.12249756],
       [ 502.22506714],
       [ 499.8972168 ],
       [ 498.03112793],
       [ 490.36291504],
       [ 492.543396  ],
       [ 489.70007324],
       [ 485.90362549],
       [ 507.79223633],
       [ 492.48773193],
       [ 499.48648071],
       [ 490.868

In [23]:
graph = tf.get_default_graph()
summary_writer = tf.summary.FileWriter("/Users/dtsmith/Temp", graph)
summary_writer.flush()

http://localhost:6006 for Tensorboard and the computation graph is ![graph3.png](graph3.png)

In [24]:
sess.close()
tf.reset_default_graph()

## Numpy Style Slicing is Not Supported!

Slicing a matrix requires reshaping into a vector and using an index vector.

## Let's Look at Linear Regression in TensorFlow

For another example, see http://tneal.org/post/tensorflow-iris/TensorFlowIris/.

In [25]:
w=tf.Variable(tf.random_uniform([4, 1]))

In [26]:
sess = tf.Session()

In [27]:
sess.run(w)

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: _send_Variable_0 = _Send[T=DT_FLOAT, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=-6966595245303910397, tensor_name="Variable:0", _device="/job:localhost/replica:0/task:0/cpu:0"](Variable)]]

In [28]:
sess.run(tf.global_variables_initializer())

In [29]:
sess.run(w)

array([[ 0.33409262],
       [ 0.46041727],
       [ 0.10946929],
       [ 0.79474235]], dtype=float32)

In [30]:
def f(X):
	return tf.matmul(X, w)

In [31]:
def objective(X, Y):
	return tf.reduce_sum(tf.square(tf.subtract(Y,f(X))))

Let's generate some synthetic data, and see how we can get a numpy array into TensorFlow.

In [32]:
a = np.array([[1, 2, 3, 4]], dtype=np.float32)
a

array([[ 1.,  2.,  3.,  4.]], dtype=float32)

In [33]:
XX = np.random.rand(10000, 4)
XX

array([[ 0.88188774,  0.67464137,  0.48440156,  0.37848818],
       [ 0.76772672,  0.84867809,  0.61436298,  0.55006959],
       [ 0.53387272,  0.51001335,  0.41807708,  0.32036248],
       ..., 
       [ 0.23904654,  0.71770961,  0.8896081 ,  0.9268231 ],
       [ 0.94013785,  0.50352542,  0.02433777,  0.49397177],
       [ 0.38520675,  0.50389889,  0.25341349,  0.68202109]])

In [34]:
YY = np.dot(XX, a.transpose())
YY

array([[ 5.19832786],
       [ 6.50845019],
       [ 4.08958055],
       ..., 
       [ 8.05058245],
       [ 3.99608907],
       [ 4.88132933]])

In [35]:
X = tf.placeholder(tf.float32, [None, 4])
Y = tf.placeholder(tf.float32, [None, 1])

**Automatic differentiation** at work here - no more calculus by hand.

In [36]:
grad = tf.gradients(objective(X,Y), [w])

Let's set up the optimization problem. We want to adjust the weights ```w``` in order to mimimize the sum of squared differences.

```python
def objective(X, Y):
    return tf.reduce_sum(tf.square(tf.subtract(Y,f(X))))
```

In [37]:
step = tf.constant(1e-5)

In [38]:
sess.run(tf.global_variables_initializer())

In [39]:
for i in range(200):
	sess.run(tf.assign_add(w, tf.multiply(-step, grad[0])), feed_dict={X:XX, Y:YY})

In [40]:
sess.run(w)

array([[ 1.06247234],
       [ 2.00820899],
       [ 2.99523067],
       [ 3.93403554]], dtype=float32)

What is ```w```? Remember, we're trying to find the weights that fit

```python
def f(X):
	return tf.matmul(X, w)
```

to ```Y```.

In [41]:
sess.run(objective(X, Y))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "//anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "//anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "//anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "//anaconda/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "//anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "//anaconda/envs/tensorflow/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 

How do we fix this? Remember, ```X``` and ```Y``` are *placeholders*, and we need to tell TensorFlow what they should be using ```feed_dict```. See http://stackoverflow.com/questions/33810990/how-to-feed-a-placeholder.

In [42]:
sess.run(objective(X, Y), feed_dict={X:XX, Y:YY})

6.9916677

# Finis

We're finished with this tutorial.

You can practice by using TensorFlow to [fit a linear regression model to housing prices](http://www.learndatasci.com/predicting-housing-prices-linear-regression-using-python-pandas-statsmodels/).

And to facilitate your TensorFlow work on GPU, you can [run notebooks on AWS](https://blog.keras.io/running-jupyter-notebooks-on-gpu-on-aws-a-starter-guide.html).

[Set up an instance with TensorFlow and Jupyter notebooks](https://medium.com/@vamsiramakrishnan/setup-a-cloud-based-machine-learning-system-from-scratch-aws-ec2-g-2x2-9216449d558d).